In [39]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime
%matplotlib inline
import arcpy

In [41]:
arcpy.env.workspace = './data/cyklo_alko.gdb/'
arcpy.env.overwriteOutput = True

In [2]:
data = pd.read_csv(r'../nehody-chodci/scratch/nehody_all.csv')

In [3]:
data.columns

Index(['X', 'Y', 'id', 'obec', 'datum', 'komunikace_druh', 'komunikace_cislo',
       'zavineni', 'ridic_kategorie', 'ridic_ovlivneni', 'vinik_alkohol',
       'ridic_stav', 'obeti_usmrceno', 'obeti_lehce', 'obeti_tezce',
       'vozidel_celkem', 'vinik_vozidlo_znacka', 'vinik_vozidlo_po_nehode',
       'skoda', 'unik_hmot', 'vinik_vozidlo_druh', 'vinik_vozidlo_rok_vyroby',
       'vinik_vozidlo_vlastnik', 'vinik_skoda_na_vozidle', 'vyprosteni',
       'nehoda_charakter', 'srazka_vozidel_druh', 'hlavni_priciny',
       'povrch_stav', 'meteo', 'rozhled', 'nehoda_poloha', 'prednost_v_jizde',
       'smerove_podminky', 'druh_krizujici_komunikace', 'smer_jizdy',
       'nehoda_druh', 'pevna_prekazka_druh', 'povrch_druh', 'komunikace_stav',
       'viditelnost', 'komunikace_deleni', 'rizeni_provozu', 'objekty',
       'misto_nehody', 'smyk'],
      dtype='object')

In [30]:
data.ridic_stav.value_counts()

dobrý -žádné nepříznivé okolnosti nebyly zjištěny      654106
jiný nepříznivý stav                                   145739
pod vlivem alkoholu, obsah alkoholu v krvi do 0,99‰     26719
pod vlivem alkoholu obsah alkoholu v krvi 1‰ a více     19396
unaven, usnul, náhlá fyzická indispozice                 6759
pod vlivem léků, narkotik                                1458
nemoc, úraz apod.                                         405
invalida                                                  365
řidič při jízdě zemřel (infarkt apod.)                    293
pokus o sebevraždu, sebevražda                             77
dtype: int64

In [12]:
alko = data[(data.vinik_vozidlo_druh == 'jízdní kolo') & (data.vinik_alkohol.str.contains('alkoholu'))]

In [16]:
alko.datum = pd.to_datetime(alko.datum)

C:\Anaconda3\lib\site-packages\pandas\core\generic.py:2177: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [19]:
#nehody v case
serie = pd.Series(np.ones(len(alko)), index=alko['datum'])
serie.resample('A', how='sum')#.plot('bar')

datum
2007-12-31    465
2008-12-31    439
2009-12-31    528
2010-12-31    534
2011-12-31    644
2012-12-31    731
2013-12-31    731
2014-12-31    794
2015-12-31    181
Freq: A-DEC, dtype: float64

In [27]:
alko14 = alko[(alko.datum >= '01-01-2014') & (alko.datum < '01-01-2015')]

In [28]:
alko14.vinik_alkohol.value_counts()

ano, obsah alkoholu v&nbsp;krvi 1,5‰ a více        533
ano, obsah alkoholu v&nbsp;krvi od 1,0‰ do 1,5‰    133
ano, obsah alkoholu v&nbsp;krvi od 0,5‰ do 0,8‰     45
ano, obsah alkoholu v&nbsp;krvi od 0,8‰ do 1,0‰     33
ano - obsah alkoholu v krvi 1‰ a více               30
ano, obsah alkoholu v&nbsp;krvi do 0,99‰            18
pod vlivem drog a alkoholu                           2
dtype: int64

In [36]:
alko14[alko14.obeti_lehce > 1]

,X,Y,id,obec,datum,komunikace_druh,komunikace_cislo,zavineni,ridic_kategorie,ridic_ovlivneni,...,nehoda_druh,pevna_prekazka_druh,povrch_druh,komunikace_stav,viditelnost,komunikace_deleni,rizeni_provozu,objekty,misto_nehody,smyk
646338,-641221.692,-1041417.007,50206141074,Hradec Králové (Královéhradecký kraj),2014-10-10 17:40:00,komunikace místní,0,řidičem nemotorového vozidla,ostatní řidiči vozidel,řidič nebyl ovlivněn,...,jiný druh nehody,"nepřichází v úvahu, nejde o srážku s pev",živice,"dobrý, bez závad","ve dne, viditelnost nezhoršená vlivem po",žádná z uvedených,žádný způsob řízení provozu,žádné nebo žádné z uvedených,mimo křižovatku,ne
659252,-620985.403,-1142349.324,60306140887,Níhov (Jihomoravský kraj),2014-09-28 20:30:00,silnice 3.třídy,3796,řidičem nemotorového vozidla,ostatní řidiči vozidel,řidič nebyl ovlivněn,...,srážka s chodcem,"nepřichází v úvahu, nejde o srážku s pev",živice,"dobrý, bez závad","v noci - bez veřejného osvětlení,viditel",dvoupruhová,žádný způsob řízení provozu,žádné nebo žádné z uvedených,mimo křižovatku,ne
666834,-434070.449,-1133397.731,70208143195,Písek (Moravskoslezský kraj),2014-06-22 19:45:00,komunikace místní,0,řidičem nemotorového vozidla,ostatní řidiči vozidel,jiné ovlivnění,...,srážka s jedoucím nekolejovým vozidlem,"nepřichází v úvahu, nejde o srážku s pev",živice,"dobrý, bez závad","ve dne, zhoršená viditelnost (svítání,so",dvoupruhová,žádný způsob řízení provozu,žádné nebo žádné z uvedených,mimo křižovatku,ne
670124,-496877.065,-1120582.534,70406140419,Suchdol nad Odrou (Moravskoslezský kraj),2014-06-06 19:15:00,silnice 3.třídy,4738,řidičem nemotorového vozidla,ostatní řidiči vozidel,řidič nebyl ovlivněn,...,srážka s jedoucím nekolejovým vozidlem,"nepřichází v úvahu, nejde o srážku s pev",živice,"dobrý, bez závad","ve dne, viditelnost nezhoršená vlivem po",dvoupruhová,místní úprava (vyplní se pol. 24),"most, nadjezd, podjezd, tunel","na křižovatce, uvnitř hranic křižovatky",ne
671613,-494864.340,-1087331.605,70606140145,Opava (Moravskoslezský kraj),2014-03-13 16:45:00,silnice 1.třídy,56,řidičem nemotorového vozidla,ostatní řidiči vozidel,oslněn sluncem,...,srážka s jedoucím nekolejovým vozidlem,"nepřichází v úvahu, nejde o srážku s pev",živice,"dobrý, bez závad","ve dne, zhoršená viditelnost (svítání,so",třípruhová,žádný způsob řízení provozu,žádné nebo žádné z uvedených,mimo křižovatku,ne
672237,-488260.200,-1089223.418,70606140772,Kravaře (Moravskoslezský kraj),2014-09-19 20:40:00,silnice 1.třídy,56,řidičem nemotorového vozidla,ostatní řidiči vozidel,řidič nebyl ovlivněn,...,srážka s jedoucím nekolejovým vozidlem,"nepřichází v úvahu, nejde o srážku s pev",živice,"dobrý, bez závad","v noci - s veřejným osvětlením,viditelno",dvoupruhová,místní úprava (vyplní se pol. 24),žádné nebo žádné z uvedených,"na křižovatce, uvnitř hranic křižovatky",ne
679238,-538152.321,-1121330.073,140506140832,Velká Bystřice (Olomoucký kraj),2014-06-28 19:05:00,komunikace místní,0,řidičem nemotorového vozidla,ostatní řidiči vozidel,řidič nebyl ovlivněn,...,srážka s jedoucím nekolejovým vozidlem,"nepřichází v úvahu, nejde o srážku s pev",živice,"dobrý, bez závad","ve dne, viditelnost nezhoršená vlivem po",dvoupruhová,žádný způsob řízení provozu,žádné nebo žádné z uvedených,mimo křižovatku,ne
682652,-559990.609,-1082806.656,140906140328,Hrabišín (Olomoucký kraj),2014-06-21 20:25:00,silnice 2.třídy,446,řidičem nemotorového vozidla,ostatní řidiči vozidel,ovlivněn jednáním jiného účastníka sil. provozu,...,srážka s jedoucím nekolejovým vozidlem,"nepřichází v úvahu, nejde o srážku s pev",živice,"dobrý, bez závad","ve dne, viditelnost nezhoršená vlivem po",dvoupruhová,žádný způsob řízení provozu,žádné nebo žádné z uvedených,mimo křižovatku,ne
684422,-568819.968,-1128219.774,141206140209,Ptení (Olomoucký kraj),2014-06-17 17:22:00,silnice 3.třídy,37349,řidičem nemotorového vozidla,ostatní řidiči vozidel,řidič nebyl ovlivněn,...,srážka s jedoucím nekolejovým vozidlem,"nepřichází v úvahu, nejde o srážku s pev",živice

In [38]:
alko.viditelnost.value_counts()

ve dne, viditelnost nezhoršená vlivem po    2995
v noci - s veřejným osvětlením,viditelno    1087
v noci - bez veřejného osvětlení,viditel     620
ve dne, zhoršená viditelnost (svítání,so     136
v noci - s veřejným osvětlením, zhoršená      91
ve dne, zhoršená viditelnost vlivem pově      67
v noci - bez veřejného osvětlení, vidite      51
dtype: int64

In [42]:
opravy = pd.read_excel('data/opravy_prodlouzeni.xlsx')

In [51]:
viditelnost = []

for row in opravy.iterrows():
    if row[1]['viditelnost'] not in viditelnost:
        viditelnost.append(row[1]['viditelnost'])
        
viditelnost = viditelnost[0:len(viditelnost)-2]


fc = 'cyklo_alko_tbl'

cursor = arcpy.UpdateCursor(fc)
for row in cursor:
    old = row.getValue("viditelnost")
    for val in viditelnost:
        if (val.startswith(old)):
            row.setValue("viditelnost", val)
            cursor.updateRow(row)
            continue

In [37]:
alko.to_csv('data/cyklo_alko.csv', encoding='utf-8', index=False)